# An Illustrative Example 

## Import Modules 

energiapy is a component-based data-driven modeling platform.

Component are categorized into:

Scope components which define the boundaries and extent of the system: 

1. Basis - Provides a scale to the data being provided 
2. Loc - A location 
3. Link - A linkage between two locations 
4. Period - A time period, collections of these make a temporal scale 

Commodities which flow through the system: 

1. Resource 
2. Material (same as Resource)
3. Land 
4. Cash (same as Econ)

Operations which perform actions that elicit flow: 

1. Process - Convert one Resource into another
2. Storage - Store Resource to withdraw at a later time 
3. Transit - Transport Resource from one location to another 

Impact which is a consequence of an Action or a Flow: 

1. Econ - Economic (same as Cash)
2. Environ - Environmental 
3. Social 


In [1]:
from energiapy.components.measure.basis import Basis
from energiapy.components.commodity.resource import Resource
from energiapy.components.commodity.misc import Cash, Land, Material
from energiapy.components.impact.categories import Econ
from energiapy.components.impact.categories import Environ
from energiapy.components.impact.categories import Social
from energiapy.components.temporal.period import Period
from energiapy.components.spatial.linkage import Link
from energiapy.components.spatial.location import Loc
from energiapy.modeling.model import Model
from energiapy.components.operation.process import Process
from energiapy.components.game.player import Player

In [2]:
m = Model()

## Basis 

These can be declared relationally 

In [3]:
m.mw = Basis(label='Megawatt')
m.kw = m.mw / 1000
m.g = Basis(label='Gram')
m.kg = 1000 * m.g
m.ton = m.kg * 1000
m.mton = m.ton * 1000
m.mile = Basis(label='Mile')
m.km = m.mile * 1.60934
m.acre = Basis(label='Acre')

.howmany() is a helpful feature to see how these are related 

In [ ]:
m.ton.howmany(m.g)

In [ ]:
m.mw.howmany(m.kw)

## Temporal Scales

These are generated based on the temporal periods declared. 

In [6]:
m.h = Period()
m.d = m.h * 24
m.y = m.d * 365
# m.w = m.d * 7
# m.s = Period()

.howmany() can be used for these as well 

In [7]:
m.h.of

In [ ]:
m.y.howmany(m.h)

These are all collected in model.time

In [ ]:
m.time.periods

The scale defined by the sparsest time period becomes the horizon of the problem 

In [ ]:
m.time.sparsest

In [ ]:
m.time.horizon

It can be directly accessed as well 

In [ ]:
m.horizon

In [ ]:
m.time.tree

In [ ]:
m.time.find(8760)

The densest discretization can also be found

In [ ]:
m.time.densest

## Locations 

Locations can be nested. For example, Houston (htown) and College Station (cstat) for Texas (tx) 

In [10]:
m.htown = Loc()
m.cstat = Loc()
m.tx = m.htown + m.cstat
m.sd = Loc()
m.la = Loc()
m.cali = m.sd + m.la
m.usa = m.tx + m.cali
m.ath = Loc()
m.amd = Loc()
m.eu = m.ath + m.amd
m.pune = Loc()
m.goa = Loc()
m.ind = m.pune + m.goa
m.earth = m.usa + m.eu + m.ind

Use the tree feature to determine how locations are nested 

In [11]:
m.usa.tree()

{cali: {la: {}, sd: {}}, tx: {cstat: {}, htown: {}}}

All spatial components are stored in model.space 

In [ ]:
m.space.locs

## Linkages 

Linkages can be declared between locations 

In [640]:
m.grid = Link(source=m.cali, sink=m.tx, dist=1400, basis=m.mile)
m.sea = Link(source=m.usa, sink=m.eu, dist=10000, basis=m.km, bi=True)

Note that locations are strictly uni directional. In the example above. m.sea_ is generated in the opposite direction

In [ ]:
m.sea.source, m.sea.sink

In [ ]:
m.sea_.source, m.sea_.sink

Determine if locations are connected 

In [ ]:
m.cali.connected(m.tx)

Print out the links using .links 

In [ ]:
m.usa.links(m.eu)

Generate a reverse link if necessary using negation 

In [ ]:
m.grid_back = -m.grid
m.grid_back.source, m.grid_back.sink

## Cash 

Cash is also economic impact (Econ)
The exchange rates can be set 

In [646]:
m.usd = Cash(m.usa)
m.eur = Cash(m.eu)
m.inr = Cash(m.ind)
m.eur == m.usd * 0.92
m.inr == m.usd / 84.08

Note that all locations with a location adopt the same currency. Texas, for example, has the currency (USD) given that it is in the US

In [ ]:
m.tx.currency

All non Scope components can be modeled 

Here we are limiting the amount of USD that can be spent

In [12]:
m.usd.spend(m.usa, m.y) >= 1e9
m.usd.pprint()

AttributeError: 'Model' object has no attribute 'usd'

If a temporal disposition is not provided. The horizon is assumed. The year in this case

In [ ]:
m.usd.spend(m.eu) <= 1e9
m.usd.pprint()

Here, we are specifying that not more than 5 usd can be spent in the EU on any particular day  

In [650]:
m.usd.spend(m.eu, m.d) <= 5

Whereas, here we are providing a different value for everyday 

In [651]:
m.usd.spend(m.tx) <= list(range(365))

The temporal disposition can be specified. But given that the length of the values is 365, the constraint is written for each day

The program for the entire model can be written out anytime 

In [ ]:
m.pprint()

The full program (with parameter values) can be printed using model.pprint(True)

In [653]:
# m.pprint(True)

Some more example constraints are provided below 

In [654]:
# m.usd.earn(m.y) > 20
# m.eur.spend(m.ath) <= 1e9
# m.eur.spend(m.amd) < 4 * 1e8
# m.usd.earn(m.htown) >= 4 * 1e5
# m.usd.earn(m.cstat) >= 1e5
# m.inr.spend() <= 4 * 1e9

All impacts are stored in model.impact 

In [ ]:
m.impact.econs

## Emission 

Similarly Emissions can be declared 

In [ ]:
m.gwp = Environ()
m.eut = Environ()
m.impact.environs

And bound 

In [657]:
# m.gwp.emit(m.eu, m.y) <= 3 * 1e5
m.gwp.emit(m.ind, m.y) <= 4 * 1e5
m.gwp.emit(m.usa, m.y) <= 3 * 1e5
m.gwp.abate(m.usa, m.y) >= 6 * 1e3

## Social Impact

Social impacts are treated in the same vein 

In [ ]:
m.mrh = Social(label='Median Risk Hours')
m.impact.socials

In [659]:
m.mrh.benefit(m.eu, m.y) <= 3 * 1e5
m.mrh.benefit(m.ind, m.y) <= 4 * 1e5
m.mrh.detriment(m.htown, m.y) <= 3 * 1e5

In [ ]:
m.mrh.pprint(True)

## Resource 

Resources are the primary commodity. It is helpful to provide a basis 

In [661]:
m.h2 = Resource(basis=m.ton, label='Hydrogen')

Every component has a positive and negative flow 

In [ ]:
m.h2.buy

In [ ]:
m.h2.sell

In [ ]:
m.h2.sell.neg

In [ ]:
m.h2.flows

Every flow has a consequence.

Which can be negative, such as expenditure on purchase 

In [ ]:
m.h2.buy.spend

Or, positive. Such as credits given to a buyer for purchasing a particular good 

In [ ]:
m.h2.buy.earn

Every flow or consequence can be bound

In [668]:
m.h2.sell(m.ind) >= 1e6

In [ ]:
m.h2.pprint()

However, providing a value (==). Makes this a calculation 

In [670]:
m.h2.buy.spend(m.usd, m.usa, m.y) == 33
m.h2.buy.emit(m.gwp) == 20

In [ ]:
m.h2.pprint()

In [672]:
# parameter
# m.h2.sell_emit(m.eut) = 10

Some more constraints 

In [673]:
# sum(p.sell(m.h2, m.ind, t) for t in p.time)
m.h2.sell() >= 3 * 1e6
m.h2.sell.spend(m.inr, m.ind) == 2
m.h2.sell.spend(m.usd, m.usa) == 2.5
# This shows the price being entered is in USD
# converted to EUR as the location is given as the EU
m.h2.sell.spend(m.usd, m.eu) == 1.5

# m.gwp.emit(m.h2) <= 1e6

m.pow = Resource(basis=m.mw, label='Power')

# m.gwp.emit(m.eu) == [5, 5, 6]
m.gwp.emit(m.ind) == list(range(365))
m.gwp.emit(m.usa) == 3

m.sun = Resource(basis=m.mw, label='Solar')
m.wind = Resource(basis=m.mw, label='Wind')
m.ng = Resource(basis=m.ton, label='Natural Gas')


m.ng.buy(m.usa) <= 1e6
m.ng.buy(m.tx) == 1e5  # tx needs to buy exactly 100,000 tons
m.ng.buy(m.eu) <= 1e6
m.ng.buy(m.ind) <= 1e6


m.o2 = Resource(basis=m.ton, label='Oxygen')
m.h2o = Resource(m.ton, label='Water')
# this applies everywhere
m.h2o.buy.spend(m.usd) == 0.5
# these are only at location
# the rest retain the global value provided
m.h2o.buy.spend(m.usd, m.usa) == 1
# m.h2o.buy.spend(m.ath) == 0.4

m.co2 = Resource(m.ton, label='Carbon Dioxide')
m.co2.sell.emit(m.gwp) == 100
m.co2.sell.emit(m.eut) == 100

## Material and Land Use

Materials and Land are essentially resources but land cannot be transported

In [674]:
m.steel = Resource(m.ton, label='Steel')
m.steel.buy.emit(m.gwp) == 10
m.cement = Resource(m.ton, label='Cement')

In [675]:
m.sand = Land(m.acre, label='Sand')
m.farm = Land(m.acre)

Their use can be bound as well 

In [676]:
m.sand.use(m.usa, m.y) <= 1e6

They can be forced to be disposed 

In [ ]:
m.steel.dispose(m.usa, m.y) >= 1e6
m.steel.pprint()

Check the flows and consequences of any component

In [ ]:
m.steel.flows

## Process 

In [679]:
m.pem = Process(label='PEM Electrolyzer', basis=m.ton)

Processes can be given a capacity limit 

In [680]:
m.pem.setup(m.usa) <= 1000

In [681]:
# m.pem.setup.use.emit(m.h2, m.gwp, m.usa) == 30

Their material and land use can be set 

In [ ]:
m.pem.setup.use(m.steel) == 3
m.pem.pprint()

Related expenditure can be provided 

In [683]:
m.pem.setup.spend(m.usd, m.usa) == 1e6
# m.pem.setup.spend(m.eur, m.eu) == list(range(8760))
m.pem.setup.spend(m.inr, m.ind) == 1e6
m.pem.operate.spend(m.usd, m.usa) == 1e3

Conversion balance can be provided for each process 

As also operational time (lag), setup time, etc. 

In [ ]:
m.pem.conv(m.h2) == -0.4 * m.pow + 0.8 * m.o2
m.pem.conv(m.h2) == -0.2 * m.h2o
m.pem.setup(m.usa) >= 10
m.pem.operate.time = 2 * m.h
# m.pem.life == 20 * m.y
# m.pem.use(m.farm) == (200, 300)
# m.pem.setup.time = 30 * m.d
# m.pem.consume(m.pow) <= 30
# m.pem.produce(m.o2) >= 40
m.pem.pprint()

In [ ]:
m.program.pem_conv.pprint(True)

In [ ]:
# m.wf = Process(label = 'Wind Farm')
# m.wf.life(m.y) = 50
# m.wf(m.pow) = 1.3*m.wind
# m.wf.capex(m.usa) = 1e6
# m.wf.capex(m.eu) = [1e6, 0.95*1e6, 0.9*1e6]
# m.wf.capex(m.ind) = 1e6
# # bounds are summed for wfs across at all locations
# m.wf.cap() >= 1000
# m.wf.cap() <= 10
# m.wf.dispose(m.y) <= [100, 200, 300]
# m.wf.op(m.tx) = # variable intermittency data a
# m.wf.op(m.cali) = # variable intermittency data b
# m.wf.op(m.ath) = # variable intermittency data c
# m.wf.use() = 1.6*m.steel + 0.3*m.cement

# m.pv = Process(label = 'Solar PV')
# m.pv(m.pow) = 1.3*m.sun
# # exacts are applied to pv at all locations
# # capacity modes
# m.pv.setup.spend(cap = (0, 20)) = 100
# m.pv.setup.spend(cap = (20, 40)) = 70
# m.pv.setup.spend(cap = (40, 80)) = 55  # check whether max cap bin here matches the max capacity
# m.pv.operate(m.tx) = # variable intermittency data a
# m.pv.operate(m.cali) = # variable intermittency data b
# m.pv.operate(m.ath) = # variable intermittency data c
# m.pv.cap() <= 80

# m.tx.has(m.wf, m.pv, m.pem)

## Storage 

This is being developed 

In [ ]:
# m.lii = Storage(m.usa, m.eu, label='Lithium Ion Battery')
# m.lii.store(m.pow)
# m.lii.cap >= 40
# m.lii.cap <= 1000

# # assumed to be available at all locations
# m.silo = Storage(label='Hydrogen Silo')
# m.silo.store(m.h2) == 0.8 * m.pow
# m.silo.cap == 50
# m.silo.capex(m.usa) == 1e6
# m.silo.capex(m.eu) == [1e6, 0.95 * 1e6, 0.9 * 1e6]

## Transit 

In [ ]:
# m.wire = Transit(m.grid, label = 'Car')
# m.wire.carry(m.pow)
# m.wire.capex(m.usa) = 1e6 # all links within USA
# # if speed not provided then instantenous

# m.ship = Transit(m.sea, label = 'Ship')
# # can make this return an attribute that gets set
# # setattr can be used to then set the value
# m.ship.carry(m.h2) == - 0.8*m.pow
# m.ship.capex(m.sea) == 1e6
# m.ship.speed(m.mile, m.h) == 10

## The model

In [ ]:
# add binaries only where needed, such as operation with minimum capacities
m.pprint()